## Setting up the notebook

In [ ]:
# installing packages
!pip install pip==24.0
!pip install numpy==1.23.5
!pip install tensorboardX
!pip install subword-nmt
!pip install sentencepiece

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with 

In [ ]:
# importing packages
import numpy
import os
import tensorboardX
import sentencepiece as spm

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
source_code = 'eng'
target_code = 'nde'

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing fairseq
#!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
!pip install --editable ./

/content/drive/MyDrive/Research/eng-to-nde/fairseq
Obtaining file:///content/drive/MyDrive/Research/eng-to-nde/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 19.2 MB/s eta 0:00:00
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-0.editable-cp310-cp310-linux_x86_64.whl size=9580 sha256=8d0c526e6719ebcf8379385d6ea8c041da82cbae8e840ca6b05484ccb7542

## Evaluate Model on Raw Test and Global Test Sets

In [ ]:
import sacrebleu

In [ ]:
before_punc = '!,.:;?)'
def remove_whitespace_before(segment, punc):
  """
  This function looks for punctuation in a segment and removes the whitespace
  before the punctuation mark.

  Args:
    segment (str): segment to be processed.
    punc (str): string of punctuation marks.

  Returns:
    The segment with whitespace removed before punctuation marks in punc.
  """
  punc_positions = []
  for i, symbol in enumerate(segment):
    if symbol in punc:
      punc_positions.append(i)

  punc_positions.reverse()
  for pos in punc_positions:
    if segment[pos-1] == " ":
      segment = segment[0:pos-1] + segment[pos:]

  return segment

### BPE

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe')

In [ ]:
!mkdir -p trained_model
!cp data-bin/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin/dict.nde.txt trained_model/dict.nde.txt
!cp data/bpe.codes.4000 trained_model/bpe.codes.4000
!cp checkpoints-bpe/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'subword_nmt' \
--bpe-codes trained_model/bpe.codes.4000 \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "D-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 13:05:51.256124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 13:05:51.280661: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 13:05:51.287741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 13:05:51.305774: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 13:05:52.764021: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'subword_nmt' \
--bpe-codes trained_model/bpe.codes.4000 \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/test.eng \
trained_model/ | grep -P "D-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 13:09:08.139529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 13:09:08.167119: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 13:09:08.176185: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 13:09:08.195454: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 13:09:09.598599: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpe/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

### ULM

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm')

In [ ]:
!mkdir -p trained_model
!cp data-bin/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin/dict.nde.txt trained_model/dict.nde.txt
!cp data/joint.model trained_model/joint.model
!cp checkpoints-ulm/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 15:30:21.684007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 15:30:21.710180: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 15:30:21.721659: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 15:30:21.743133: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 15:30:23.135292: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/test.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 15:33:39.022456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 15:33:39.047216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 15:33:39.054484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 15:33:39.072485: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 15:33:40.404681: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
!sacrebleu ref1.nde ref2.nde ref3.nde ref4.nde -i $translations_path -m bleu --force
!sacrebleu ref1.nde ref2.nde ref3.nde ref4.nde -i $post_translations_path -m bleu

{
 "name": "BLEU",
 "score": 9.6,
 "signature": "nrefs:4|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "43.0/16.6/6.6/2.4 (BP = 0.925 ratio = 0.928 hyp_len = 6480 ref_len = 6984)",
 "nrefs": "4",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}
{
 "name": "BLEU",
 "score": 9.6,
 "signature": "nrefs:4|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "43.0/16.6/6.6/2.4 (BP = 0.925 ratio = 0.928 hyp_len = 6480 ref_len = 6984)",
 "nrefs": "4",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}


In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulm/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
!sacrebleu test.nde -i $translations_path -m bleu --force
!sacrebleu test.nde -i $post_translations_path -m bleu

{
 "name": "BLEU",
 "score": 11.3,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "40.7/16.9/8.3/4.1 (BP = 0.908 ratio = 0.912 hyp_len = 37172 ref_len = 40769)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}
{
 "name": "BLEU",
 "score": 11.3,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "40.7/16.9/8.3/4.1 (BP = 0.908 ratio = 0.912 hyp_len = 37172 ref_len = 40769)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}


### BPE-Dropout

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP')

In [ ]:
!mkdir -p trained_model
!cp data-bin-25/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin-25/dict.nde.txt trained_model/dict.nde.txt
!cp /content/drive/MyDrive/Research/eng-to-nde/bpe/data/bpe.codes.4000 trained_model/bpe.codes.4000
!cp checkpoints-bpeDROP/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'subword_nmt' \
--bpe-codes trained_model/bpe.codes.4000 \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "D-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 13:24:33.230691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 13:24:33.261016: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 13:24:33.268518: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 13:24:33.286241: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 13:24:34.674713: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'subword_nmt' \
--bpe-codes trained_model/bpe.codes.4000 \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/test.eng \
trained_model/ | grep -P "D-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 13:27:52.822534: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 13:27:52.866749: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 13:27:52.879494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 13:27:52.909166: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 13:27:54.967536: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/bpeDROP/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
!sacrebleu test.nde -i $translations_path -m bleu --force
!sacrebleu test.nde -i $post_translations_path -m bleu

{
 "name": "BLEU",
 "score": 9.5,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "40.0/15.7/7.0/3.1 (BP = 0.876 ratio = 0.883 hyp_len = 35994 ref_len = 40769)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}
{
 "name": "BLEU",
 "score": 9.5,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.4.3",
 "verbose_score": "40.0/15.7/7.0/3.1 (BP = 0.876 ratio = 0.883 hyp_len = 35994 ref_len = 40769)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.4.3"
}


### ULM with Subword Regularization

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR')

In [ ]:
!mkdir -p trained_model
!cp data-bin-25/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin-25/dict.nde.txt trained_model/dict.nde.txt
!cp /content/drive/MyDrive/Research/eng-to-nde/ulm/data/joint.model trained_model/joint.model
!cp checkpoints-ulmSR/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 19:33:31.247757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 19:33:31.293323: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 19:33:31.306802: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 19:33:31.336926: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 19:33:33.000300: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/test.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 19:37:37.476406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 19:37:37.518039: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 19:37:37.529873: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 19:37:37.559526: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 19:37:39.211357: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/ulmSR/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

### Task-specific Tokenization

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok')

In [ ]:
!mkdir -p trained_model
!cp data-bin-nmt/dict.eng.txt trained_model/dict.eng.txt
!cp data-bin-nmt/dict.nde.txt trained_model/dict.nde.txt
!cp /content/drive/MyDrive/Research/eng-to-nde/ulm/data/joint.model trained_model/joint.model
!cp checkpoints-task-nmt/checkpoint_best.pt trained_model/model.pt

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/clean_gtest.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_gtest

2024-11-06 13:43:54.960476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 13:43:54.985439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 13:43:54.992709: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 13:43:55.012309: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 13:43:56.362278: W tensorflow/compiler/tf2

In [ ]:
!fairseq-interactive \
--path trained_model/model.pt \
--source-lang eng \
--target-lang nde \
--bpe 'sentencepiece' \
--sentencepiece-model trained_model/joint.model \
--beam 5 \
--lenpen 1 \
--seed 2024 \
--max-len-a 0 \
--max-len-b 100 \
--quiet \
--remove-bpe sentencepiece \
--buffer-size 100 \
--input /content/drive/MyDrive/Research/eng-to-nde/cleaned-data/test.eng \
trained_model/ | grep -P "H-[0-9]+" | cut -f3 > trained_model/translations_test

2024-11-06 13:47:09.781781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 13:47:09.806120: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 13:47:09.813049: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 13:47:09.830306: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 13:47:11.194933: W tensorflow/compiler/tf2

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/trained_model/translations_gtest'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/trained_model/post_process_translations_gtest'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')

In [ ]:
translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/trained_model/translations_test'

In [ ]:
# post-processing predictions
with open(translations_path, 'r', encoding='utf-8') as f:
  segments = f.read().splitlines()

for i in range(len(segments)):
  segments[i] = remove_whitespace_before(segments[i], before_punc)

post_translations_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/trained_model/post_process_translations_test'
with open(post_translations_path, 'w') as f:
  for segment in segments:
    f.write(segment + '\n')